## Load an LTSF-Linear Model

Train the model through 'electricity.sh' bash script.
Load an .pth file from the checkpoints folder.

In [2]:
import torch
from torchviz import make_dot
import pandas as pd
import sys
sys.path.append("..")
from models.DLinear import Model as DLinear
from models.Linear import Model as Linear
from models.NLinear import Model as NLinear


class Configs:
    def __init__(self, seq_len, pred_len):
      self.seq_len = seq_len
      self.pred_len = pred_len
      self.enc_in = 321
      self.individual = True

model = 'Linear'
# target = '40'
seq_len = 192
pred_len = 24

configs = Configs(seq_len, pred_len)
match model:
  case 'Linear':
    circuit = Linear(configs)
  case 'DLinear':
    circuit = DLinear(configs)
  case 'NLinear':
    circuit = NLinear(configs)

# basepath = '../checkpoints/Electricity_{}_{}_{}_custom_ftS_tg{}_sl{}_ll48_pl{}_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_Exp_0/checkpoint.pth'
check_point_model = '../checkpoints/Electricity_192_24_Linear_custom_ftM_sl192_ll48_pl24_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_Exp_0/checkpoint.pth'


state_dict = torch.load(check_point_model)
circuit.load_state_dict(state_dict)

total_params = sum(
	param.numel() for param in circuit.parameters()
)

print(circuit, 'with {} parameters'.format(total_params))

df = pd.read_csv('../dataset/electricity.csv')
# Load the last seq_len entries data as input and converts to tensor
x = torch.tensor(df[-seq_len:].drop(labels=['date'], axis=1).values, requires_grad=True).resize(1, seq_len, len(df.columns) - 1).float()
# Flips the neural net into inference mode
circuit.eval()

y = circuit(x)
print(y)

Model(
  (Linear): ModuleList(
    (0-320): 321 x Linear(in_features=192, out_features=24, bias=True)
  )
) with 1486872 parameters


/mnt/zkpet/venv/lib/python3.11/site-packages/torch/_tensor.py:836: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


tensor([[[  10.5662,   81.9392,    7.4823,  ..., 2459.0837,  602.2404,
          2610.2004],
         [  10.6349,   76.0962,    7.8692,  ..., 2268.2991,  435.5055,
          2588.4624],
         [  10.5574,   71.5558,    8.1297,  ..., 2088.3269,  273.8727,
          2635.1079],
         ...,
         [  11.2245,   87.8995,    8.0196,  ..., 1684.2798,  147.4261,
          2536.6782],
         [  10.3803,   84.7001,    8.0491,  ..., 2226.7456,  233.7020,
          2515.1611],
         [  10.5372,   79.9100,    8.1474,  ..., 2274.9666,  210.1272,
          2471.6387]]], grad_fn=<CopySlices>)


## ZK Inference

#### Define Files Path

In [3]:
import tracemalloc
import os
import json
from timeit import default_timer as timer

model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.ezkl')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')
srs_path = os.path.join('kzg.srs')
witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')
proof_path = os.path.join('test.pf')
sol_code_path = os.path.join('verify.sol')
abi_path = os.path.join('verify.abi')

#### Convert Model to ONNX

In [4]:
# Model was trained by 'electricity.sh' and stored into the checkpoint state 'checkpoint.pth'.
# Now we need to export the onnx file from this state file with model inputs.

# Export the model
torch.onnx.export(circuit,               # model being run
                  x,                   # model input (or a tuple for multiple inputs)
                  model_path,            # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=15,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

# Serialize data into file:
json.dump( data, open(data_path, 'w' ))

input_size = os.stat(data_path).st_size / 1024
onnx_size = os.stat(model_path).st_size / 1024
print("Input.json size: {}KB".format(input_size))
print("network.onnx size: {}KB".format(onnx_size))

Input.json size: 433.103515625KB
network.onnx size: 7601.50390625KB


### Setting circuit parameters

In [14]:
import ezkl

run_args = ezkl.PyRunArgs()
run_args.input_visibility = "private"
run_args.output_visibility = "public"
run_args.param_visibility = "fixed"
run_args.variables = [("batch_size", 1)]
# run_args.logrows = 20

try:
    table_string = ezkl.table(model_path, run_args)
#     print(table_string)
except Exception as e:
    print(f"An error occurred: {e}")

In [5]:
res = ezkl.gen_settings(model_path, settings_path, py_run_args=run_args)
assert res == True

res = ezkl.calibrate_settings(data_path, model_path, settings_path, "resources")
assert res == True

using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using column duplication for 1 fixed columns
using colu

### Compiling the Model

In [8]:
tracemalloc.start()
start = timer()

res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

end = timer()
print("Compilation time used: {} seconds.".format(end - start))

curr_size, peak_size = tracemalloc.get_traced_memory()
print("Current memory size {} KB, peak memory size {} KB".format(round(curr_size / 1024, 3),
                                                              round(peak_size / 1024, 3)))

Compilation time used: 8.938970184004575 seconds.
Current memory size 1133.075 KB, peak memory size 1158.369 KB


### Creating the circuit

In [13]:
# Setup is performed by the application developer, 
# who then deploys the resulting artifacts to production.

# get public srs from kzg ceremony, saved to srs path. 
res = ezkl.get_srs(srs_path=srs_path, settings_path=settings_path)

# setup the circuit and make sure the keys are generated afterwards. 
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        srs_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)

thread '<unnamed>' panicked at /root/.cargo/git/checkouts/halo2-049b997cf7195aea/4d7e6dd/halo2_proofs/src/poly/domain.rs:55:9:
extended_k (29, k=26, j=6) with degree (335544320) must be <= S (28), the size of the field


PanicException: extended_k (29, k=26, j=6) with degree (335544320) must be <= S (28), the size of the field

### Making a proof

In [ ]:
# Prove, invoked with ezkl prove at the cli or ezkl.prove() in Python, is called by the prover, often on the client.

# the witness data for the claim: an (input, output) pair (x,y) such that model(input) = output.
# this pair can be produced from x using the gen-witness command.
# now generate the witness file 

tracemalloc.start()
start = timer()
res = ezkl.gen_witness(
        data_path, 
        compiled_model_path, 
        witness_path
      )
assert os.path.isfile(witness_path)

# GENERATE A PROOF

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        "single",
        srs_path
    )

# print(res)
# assert os.path.isfile(proof_path)

end = timer()
prove_time = end - start
print("time used: {} seconds".format(prove_time))

snapshot = tracemalloc.take_snapshot()
top_stats = snapshot.statistics('lineno')
print("[ Top 10 ]")
for stat in top_stats[:10]:
    print(stat)

proof_size = os.stat(proof_path).st_size / 1024
print("{} size: {}KB".format(proof_path, proof_size))

spawning module 2
spawning module 3


time used: 4.010687675327063 seconds
[ Top 10 ]
/mnt/zkpet/venv/lib/python3.10/site-packages/pandas/core/internals/managers.py:2200: size=64.6 MiB, count=6, average=10.8 MiB
/mnt/zkpet/venv/lib/python3.10/site-packages/pandas/io/parsers/c_parser_wrapper.py:234: size=1753 KiB, count=26424, average=68 B
/tmp/ipykernel_1398943/790744412.py:5: size=943 KiB, count=6402, average=151 B
/tmp/ipykernel_1398943/790744412.py:10: size=73.6 KiB, count=1241, average=61 B
/usr/lib/python3.10/tracemalloc.py:558: size=68.1 KiB, count=1295, average=54 B
/mnt/zkpet/venv/lib/python3.10/site-packages/IPython/core/compilerop.py:174: size=66.9 KiB, count=739, average=93 B
/usr/lib/python3.10/posixpath.py:373: size=63.5 KiB, count=510, average=128 B
/usr/lib/python3.10/tracemalloc.py:67: size=58.1 KiB, count=929, average=64 B
/usr/lib/python3.10/tracemalloc.py:505: size=52.2 KiB, count=948, average=56 B
/usr/lib/python3.10/abc.py:123: size=48.9 KiB, count=534, average=94 B
test.pf size: 22.4736328125KB


### Verify

#### VERIFY off-chain

In [ ]:
start = timer()
res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        srs_path,
    )

assert res == True
print("verified")
end = timer()
print("time used: {} seconds".format(end - start))

verified
time used: 0.02605723962187767 seconds


#### VERIFY on-chain

In [ ]:
# Create verifier contract
res = ezkl.create_evm_verifier(
        vk_path,
        srs_path,
        settings_path,
        sol_code_path,
        abi_path,
    )
verifier_size = os.stat(sol_code_path).st_size / 1024
print("{} size: {}KB".format(sol_code_path, verifier_size))

verify.sol size: 58.140625KB


In [ ]:
# Deploy the verifier contract onchain
sol_code_path = os.path.join("verify.sol")
address_path = os.path.join('contractAddr.txt')
# assuming anvil is running
res = ezkl.deploy_evm(
    address_path,
    sol_code_path,
    'http://127.0.0.1:3030'
)

assert res == True

In [ ]:
# Verify proof onchain

with open(address_path, 'r') as f:
  addr = f.readline()

res = ezkl.verify_evm(
    proof_path,
    addr,
    'http://127.0.0.1:3030'
)

assert res == True

## Stats

In [ ]:
import numpy as np
from subprocess import Popen, PIPE

input_size = np.round(os.stat(data_path).st_size / 1024, 3)
onnx_size = np.round(os.stat(model_path).st_size / 1024, 3)
print("{} size: {}KB".format(data_path, input_size))
print("{} size: {}KB".format(model_path, onnx_size))
proof_size = np.round(os.stat(proof_path).st_size / 1024, 3)
print("{} size: {}KB".format(proof_path, proof_size))
print("prove time used: {} seconds".format(np.round(prove_time, 3)))
verifier_size = np.round(os.stat(sol_code_path).st_size / 1024, 3)
print("{} size: {}KB".format(sol_code_path, verifier_size))

p = Popen(["solc", "--bin", "--optimize", "verify.sol"], stdin=PIPE, stdout=PIPE, stderr=PIPE)
output, err = p.communicate(b"input data that is passed to subprocess' stdin")
verifier_code_size = np.round(sys.getsizeof((output.split(b'\n')[3])) / 1000, 3)
print("Verifier bytecode size: {}KB".format(verifier_code_size))

input.json size: 5.643KB
network.onnx size: 68.064KB
test.pf size: 22.474KB
prove time used: 4.011 seconds
verify.sol size: 58.141KB
Verifier bytecode size: 13.741KB
